In [1]:
import requests
import pandas as pd
import datetime
from tqdm import tqdm
import time

In [3]:
total_meetings = requests.get("https://api.openf1.org/v1/meetings").json()
session_dict = {}
lap_dict = {}
for meeting in tqdm(total_meetings):
    mk = meeting['meeting_key']
    session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    while (session_dict[mk].status_code != 200):
        time.sleep(0.5)
        session_dict[mk] = requests.get(f"https://api.openf1.org/v1/sessions?meeting_key={mk}")
    session_dict[mk] = session_dict[mk].json()
    for session in session_dict[mk]:
        if session['session_type'] == 'Race':
            session_key = session['session_key']
            lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            while (lap_dict[mk].status_code != 200):
                time.sleep(0.5)
                lap_dict[mk] = requests.get(f"https://api.openf1.org/v1/laps?session_key={session_key}")
            lap_dict[mk] = lap_dict[mk].json()
            

100%|██████████| 32/32 [01:10<00:00,  2.22s/it]


In [4]:
for elem in lap_dict.keys():
    print(elem in session_dict.keys(), elem)

True 1141
True 1142
True 1143
True 1207
True 1208
True 1210
True 1211
True 1212
True 1213
True 1214
True 1215
True 1216
True 1217
True 1218
True 1219
True 1220
True 1221
True 1222
True 1223
True 1224
True 1225
True 1226
True 1229
True 1230
True 1231
True 1232
True 1233
True 1234
True 1235
True 1236
True 1237


In [3]:
def get_total_df(lst_dict):
    ret = []
    for k, v in lst_dict.items():
        ret += v
    return pd.DataFrame(ret)

total_st_speed_df = get_total_df(lap_dict)
total_st_speed_df = total_st_speed_df.loc[:, ["meeting_key", "session_key", "driver_number", "st_speed", "lap_number"]]
total_st_speed_df.to_csv("st_speed.csv", index = False)
total_st_speed_df.groupby(['meeting_key', 'session_key', 'driver_number']).max().loc[:, "st_speed"]

meeting_key  session_key  driver_number
1141         7953         1                303.0
                          2                328.0
                          4                325.0
                          10               327.0
                          11               330.0
                                           ...  
1237         9531         44               334.0
                          55               335.0
                          63               333.0
                          77               337.0
                          81               329.0
Name: st_speed, Length: 610, dtype: float64

In [19]:
# 문자열을 datetime 객체로 변환하는 함수
def parse_datetime(date_str):
    if '.' in date_str:
        # 소수점이 있는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%f%z")
    else:
        # 소수점이 없는 경우
        return datetime.datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S%z")

total_session_df = get_total_df(session_dict)
total_session_df = total_session_df.loc[:, ["meeting_key", "session_key", "session_type", "date_start"]]
total_session_df['date_start'] = total_session_df['date_start'].map(parse_datetime)
total_session_df

,meeting_key,session_key,session_type,date_start
0,1140,9222,Practice,2023-02-23 07:00:00+00:00
1,1140,7763,Practice,2023-02-24 07:00:00+00:00
2,1140,7764,Practice,2023-02-25 07:00:00+00:00
3,1141,7765,Practice,2023-03-03 11:30:00+00:00
4,1141,7766,Practice,2023-03-03 15:00:00+00:00
...,...,...,...,...
152,1237,9524,Practice,2024-06-07 17:30:00+00:00
153,1237,9525,Practice,2024-06-07 21:00:00+00:00
154,1237,9526,Practice,2024-06-08 16:30:00+00:00
155,1237,9527,Qualifying,2024-06-08 20:00:00+00:00


In [23]:
total_meeting_df = pd.DataFrame(total_meetings)
total_meeting_df = total_meeting_df.loc[:, ["meeting_key", "meeting_name", "country_name", "circuit_short_name", "location", "year"]]
total_meeting_df

,meeting_key,meeting_name,country_name,circuit_short_name,location,year
0,1140,Pre-Season Testing,Bahrain,Sakhir,Sakhir,2023
1,1141,Bahrain Grand Prix,Bahrain,Sakhir,Sakhir,2023
2,1142,Saudi Arabian Grand Prix,Saudi Arabia,Jeddah,Jeddah,2023
3,1143,Australian Grand Prix,Australia,Melbourne,Melbourne,2023
4,1207,Azerbaijan Grand Prix,Azerbaijan,Baku,Baku,2023
5,1208,Miami Grand Prix,United States,Miami,Miami,2023
6,1210,Monaco Grand Prix,Monaco,Monte Carlo,Monaco,2023
7,1211,Spanish Grand Prix,Spain,Catalunya,Barcelona,2023
8,1212,Canadian Grand Prix,Canada,Montreal,Montréal,2023
9,1213,Austrian Grand Prix,Austria,Spielberg,Spielberg,2023
